In [1]:
import pandas as pd
from plyer import notification
import datetime
import requests
import schedule
import time

In [2]:
def alert(level, database, step):

    if level not in [1,2,3]:
        print(f"Alert level [{level}] not recognized, must be 1, 2 or 3.")

    else:

        if level == 1:
            alertTitle = "Attention: Low Alert"

        elif level == 2:
            alertTitle = "Attention: Medium Alert"

        else:
            alertTitle = "ATTENTION: HIGH ALERT !!"

        alertDate = datetime.datetime.now()
        alertMessage = f"\nFailed to load database [{database}] at the step [{step}]\nTime of Alert: [{alertDate}]"
    
        notification.notify(title = alertTitle,
                            message = alertMessage,
                            timeout = 10)


In [3]:
POKEMON_URL = "https://pokeapi.co/api/v2/pokemon?offset=0&limit=1302"
BRASIL_BANKS_URL = "https://brasilapi.com.br/api/banks/v1"
COUNTRIES_URL = "https://restcountries.com/v3.1/all"

try:
    response = requests.get(COUNTRIES_URL)

    if response.status_code == 200:
        jsonResponse = response.json()
        countries_df = pd.DataFrame(jsonResponse)

    else:
        print("Could not get response, Status Code:", response.status_code)
        alert(3, "countries_dataframe", "API URL Get")
        
except requests.exceptions.RequestException as err:
    print(err)
    alert(3, "countries_dataframe", "API URL Get")

In [4]:
countries_df.head(5)

,name,tld,cca2,ccn3,cca3,cioc,independent,status,unMember,currencies,...,fifa,car,timezones,continents,flags,coatOfArms,startOfWeek,capitalInfo,postalCode,borders
0,"{'common': 'Cyprus', 'official': 'Republic of ...",[.cy],CY,196,CYP,CYP,True,officially-assigned,True,"{'EUR': {'name': 'Euro', 'symbol': '€'}}",...,CYP,"{'signs': ['CY'], 'side': 'left'}",[UTC+02:00],[Europe],"{'png': 'https://flagcdn.com/w320/cy.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [35.17, 33.37]}","{'format': '####', 'regex': '^(\d{4})$'}",NaN
1,"{'common': 'Eritrea', 'official': 'State of Er...",[.er],ER,232,ERI,ERI,True,officially-assigned,True,"{'ERN': {'name': 'Eritrean nakfa', 'symbol': '...",...,ERI,"{'signs': ['ER'], 'side': 'right'}",[UTC+03:00],[Africa],"{'png': 'https://flagcdn.com/w320/er.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [15.33, 38.93]}",NaN,"[DJI, ETH, SDN]"
2,"{'common': 'Liberia', 'official': 'Republic of...",[.lr],LR,430,LBR,LBR,True,officially-assigned,True,"{'LRD': {'name': 'Liberian dollar', 'symbol': ...",...,LBR,"{'signs': ['LB'], 'side': 'right'}",[UTC],[Africa],"{'png': 'https://flagcdn.com/w320/lr.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [6.3, -10.8]}","{'format': '####', 'regex': '^(\d{4})$'}","[GIN, CIV, SLE]"
3,"{'common': 'Bermuda', 'official': 'Bermuda', '...",[.bm],BM,060,BMU,BER,False,officially-assigned,False,"{'BMD': {'name': 'Bermudian dollar', 'symbol':...",...,BER,"{'signs': ['GB'], 'side': 'left'}",[UTC-04:00],[North America],"{'png': 'https://flagcdn.com/w320/bm.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [32.28, -64.78]}","{'format': '@@ ##', 'regex': '^([A-Z]{2}\d{2})$'}",NaN
4,"{'common': 'Vatican City', 'official': 'Vatica...",[.va],VA,336,VAT,NaN,True,officially-assigned,True,"{'EUR': {'name': 'Euro', 'symbol': '€'}}",...,NaN,"{'signs': ['V'], 'side': 'right'}",[UTC+01:00],[Europe],"{'png': 'https://flagcdn.com/w320/va.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [41.9, 12.45]}",NaN,[ITA]


In [5]:
countries_df.columns

Index(['name', 'tld', 'cca2', 'ccn3', 'cca3', 'cioc', 'independent', 'status',
       'unMember', 'currencies', 'idd', 'capital', 'altSpellings', 'region',
       'subregion', 'languages', 'translations', 'latlng', 'landlocked',
       'area', 'demonyms', 'flag', 'maps', 'population', 'gini', 'fifa', 'car',
       'timezones', 'continents', 'flags', 'coatOfArms', 'startOfWeek',
       'capitalInfo', 'postalCode', 'borders'],
      dtype='object')

In [6]:
try:

    countries_name = [resp["common"] for resp in countries_df["name"]]
    is_un_member = [resp for resp in countries_df["unMember"]]
    population = [resp for resp in countries_df["population"]]

    continents = [resp for resp in countries_df["continents"]]
    borders = [resp for resp in countries_df["borders"]]

    region = [resp for resp in countries_df["region"]]
    subregion = [resp for resp in countries_df["subregion"]]
    timezones = [resp for resp in countries_df["timezones"]]

    social_status_df = pd.DataFrame({
        "Name":countries_name,
        "Is UN member":is_un_member,
        "Population":population
    })

    borders_df = pd.DataFrame({
        "Name":countries_name,
        "Continent":continents,
        "Border with":borders
    })

    region_time_df = pd.DataFrame({
        "Name":countries_name,
        "Region":region,
        "Subregion":subregion,
        "Timezones":timezones
    })

except KeyError as err:
    print("Column not found:", err)
    alert(2, "countries_dataframe", "Data sub-df creation")


In [7]:
social_status_df

,Name,Is UN member,Population
0,Cyprus,True,1207361
1,Eritrea,True,5352000
2,Liberia,True,5057677
3,Bermuda,False,63903
4,Vatican City,True,451
...,...,...,...
245,South Sudan,True,11193729
246,Honduras,True,9904608
247,Saint Vincent and the Grenadines,True,110947
248,Sri Lanka,True,21919000


In [8]:
borders_df

,Name,Continent,Border with
0,Cyprus,[Europe],NaN
1,Eritrea,[Africa],"[DJI, ETH, SDN]"
2,Liberia,[Africa],"[GIN, CIV, SLE]"
3,Bermuda,[North America],NaN
4,Vatican City,[Europe],[ITA]
...,...,...,...
245,South Sudan,[Africa],"[CAF, COD, ETH, KEN, SDN, UGA]"
246,Honduras,[North America],"[GTM, SLV, NIC]"
247,Saint Vincent and the Grenadines,[North America],NaN
248,Sri Lanka,[Asia],[IND]


In [9]:
region_time_df

,Name,Region,Subregion,Timezones
0,Cyprus,Europe,Southern Europe,[UTC+02:00]
1,Eritrea,Africa,Eastern Africa,[UTC+03:00]
2,Liberia,Africa,Western Africa,[UTC]
3,Bermuda,Americas,North America,[UTC-04:00]
4,Vatican City,Europe,Southern Europe,[UTC+01:00]
...,...,...,...,...
245,South Sudan,Africa,Middle Africa,[UTC+03:00]
246,Honduras,Americas,Central America,[UTC-06:00]
247,Saint Vincent and the Grenadines,Americas,Caribbean,[UTC-04:00]
248,Sri Lanka,Asia,Southern Asia,[UTC+05:30]
